In [1]:
from functions.prompt_maker import make_one_prompt
from functions.SPARQL_executer import execute_one_query
from functions.SPARQL_generator import generate_one_sparql

selected_db = "uniprot"
user_input = "What is the UniProt ID for the protein associated with the ACE2 gene?"
user_prompt = make_one_prompt(database=selected_db, user_question=user_input)
sparql_query = generate_one_sparql(database=selected_db, user_prompt=user_prompt, verbose=True)

KeyError: 'PATH_DIR'

In [9]:
import requests

API_BASE_URL  = "http://localhost:8000"
user_input = "Behavioral abnormalities in the Fmr1 KO2 Mouse Model of Fragile X Syndrome"
payload = {
    "user_input": user_input
}
try:
    response = requests.post(
        f"{API_BASE_URL}/huflair2/?user_input=" + user_input,
    )
    response.raise_for_status()
    print(response.json())
except requests.exceptions.HTTPError as err:
    print(err)

Behavioral abnormalities in the ncbigene:108684022 (uniprot_ncbigene) KO2 Mus musculus (taxonomy_scientific_name) Model of Fragile X Syndrome


In [6]:
import json
import re

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint = "https://sparql.uniprot.org/"

query_text = """

PREFIX core: <http://purl.uniprot.org/core/>
PREFIX chembl: <http://rdf.ebi.ac.uk/terms/chembl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?UniProt
WHERE {
    VALUES ?uniprot_gene_name { "ACE2" }
    VALUES ?UniProt__class { core:Protein chembl:UniprotRef }
    ?UniProt a ?UniProt__class ;
        core:encodedBy / skos:prefLabel ?uniprot_gene_name .
}
LIMIT 100
"""

sparql = SPARQLWrapper(endpoint)

sparql.setQuery(query_text)
results = sparql.query().convert()
# results to dict
results = results.decode('utf-8')

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'<!DOCTYPE html SYSTEM "about:legacy-compat">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head><title>Error</title><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"/><link href="/" rel="home"/><link href="/base.css" type="text/css" rel="stylesheet"/><script src="https://cdn.jsdelivr.net/npm/yasgui-yasqe@2.11.22/dist/yasqe.bundled.min.js"></script><link type="text/css" rel="stylesheet" href="https://cdn.jsdelivr.net/yasqe/2.2/yasqe.min.css"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="no-referrer" name="referrer"/></head><body typeof="schema:WebPage"><a accesskey="2" href="#content" hidden="hidden">Skip Header</a><main class="sparql error" id="content"><h1>500 Internal Server Error</h1><p>The server was not able to handle your request.</p><p> If this error occurs repeatedly, \n\t\tplease <a id="contac

In [11]:
import requests

endpoint = "https://rdfportal.org/sib/sparql"

query_text = """
PREFIX core: <http://purl.uniprot.org/core/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?UniProt
WHERE {
    VALUES ?uniprot_gene_name { "ACE2" }
    ?UniProt a core:Protein ;
        core:encodedBy / skos:prefLabel ?uniprot_gene_name .
}
LIMIT 100
"""

# HTTPリクエスト送信
params = {
    'query': query_text,
    'format': 'json'
}

try:
    response = requests.get(endpoint, params=params, timeout=600)
    response.raise_for_status()  # エラー発生時に例外を投げる
    results = response.json()
    print("Results:", results)
except requests.exceptions.RequestException as e:
    print("Error:", e)


Error: Expecting value: line 1 column 1 (char 0)


In [9]:
results = None

In [12]:
results["results"]["bindings"]

TypeError: 'NoneType' object is not subscriptable

In [14]:
from flair.data import Sentence
from flair.models import EntityMentionLinker
from flair.nn import Classifier

sentence = Sentence("Behavioral abnormalities in the Fmr1 KO2 Mouse Model of Fragile X Syndrome")

In [8]:
species_linker = EntityMentionLinker.load("species-linker")

In [9]:
gene_linker = EntityMentionLinker.load("gene-linker")

In [15]:
tagger = Classifier.load("hunflair2")

2024-12-19 12:27:27,509 SequenceTagger predicts: Dictionary with 21 tags: O, S-Chemical, B-Chemical, E-Chemical, I-Chemical, S-Gene, B-Gene, E-Gene, I-Gene, S-Disease, B-Disease, E-Disease, I-Disease, S-Species, B-Species, E-Species, I-Species, S-CellLine, B-CellLine, E-CellLine, I-CellLine


In [16]:
tagger.predict(sentence)
gene_linker.predict(sentence)
species_linker.predict(sentence)

In [17]:
for entity in sentence.get_labels("link"):
    print(entity)

Span[4:5]: "Fmr1" → 108684022/name=FRAXA (219.9510040283203)
Span[6:7]: "Mouse" → 10090/name=Mus musculus (213.6201934814453)
